In [5]:
import time
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re

# 抓取單個earnings call transcripts

In [ ]:
col = ['stock_symbol', 'title', 'date', 'time', 
          '1_day_close', '2_day_close', '3_day_close', '4_day_close', '5_day_close', '7_day_close', '14_day_close', 
          '1_month_close', '2_month_close']

In [17]:
import pandas as pd
# 定義欄位名稱
col = ['stock_symbol', 'title', 'date', 'paragraphs']

# 建立一個空的 DataFrame，並設定欄位名稱
df = pd.DataFrame(columns=col)
df

,stock_symbol,title,date,paragraphs


In [44]:
# 初始化 WebDriver
driver = webdriver.Chrome()
# <selenium.webdriver.chrome.webdriver.WebDriver (session="935d0314ffa851b3fafde32c703aa0a9")>
base_url = "https://www.fool.com/earnings-call-transcripts/"
driver.get(base_url)

# 等待並點擊「I Accept」按鈕
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
    accept_button.click()
    print("Clicked 'I Accept' button.")
except Exception as e:
    print(f"Error clicking 'I Accept' button: {e}")

    
# 點擊 "Load More" 按鈕三次
for i in range(3):
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(text(),'Load More')]")))
        driver.execute_script("arguments[0].click();", load_more_button)
        print(f"Clicked 'Load More' button {i+1} time(s).")
        time.sleep(3)  # 等待页面加载
    except Exception as e:
        print("Error:", e)
        break    
# 等待並點擊彈出視窗的關閉按鈕
# try:
#     close_popup_button = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable((By.CLASS_NAME, "wf-popup-modal--close"))
#     )
#     close_popup_button.click()
#     print("Closed popup window.")
# except Exception as e:
#     print(f"Error closing popup window: {e}")

# 已處理過url
processed_urls = set()
transcript_links = driver.find_elements(By.CSS_SELECTOR, ".text-gray-1100 a")
link = transcript_links[59]
print('len(transcript_links): ',len(transcript_links))
transcript_url = link.get_attribute("href")
print('transcript_url: ', transcript_url)

try:
    transcript_response = requests.get(transcript_url)
    transcript_soup = BeautifulSoup(transcript_response.text, "html.parser")
    
    # 抓取 header 標籤中的 h1 標籤內容
    headers = transcript_soup.find_all('header')
    if len(headers) > 1:
        header_h1 = headers[1].find('h1')
        if header_h1:
            header_text = header_h1.get_text(strip=True)
            
            # 抓取股票代號 stock_symbol
            match = re.search(r'\((.*?)\)', header_text)
            if match:
                stock_symbol = match.group(1)
                print('stock_symbol:',stock_symbol)
            else:
                print("沒有找到括號內的文字")
                
        else:
            header_text = "No h1 found in the second header"
    else:
        header_text = "Less than 2 headers found"
    print('header_text:', header_text)
    
    # 抓取 <span id="date"> 和 <em id="time"> 標籤內容
    date_span = transcript_soup.find('span', id='date')
    time_em = transcript_soup.find('em', id='time')
    date_text = date_span.get_text(strip=True) if date_span else "No date found"
    time_text = time_em.get_text(strip=True) if time_em else "No time found"
    
    print('date_text:', date_text)
    print('time_text:', time_text)
    print('我是分隔線-------------------------------------')

    # 抓取所有 <p> 標籤內容
    paragraphs = transcript_soup.find_all('p')
    transcript_text = "\n\n".join(paragraph.get_text(strip=True) for paragraph in paragraphs)
    print('transcript_text', transcript_text)
except Exception as e:
    print(f"Error processing URL {len(processed_urls)}: {e}")
print('我是分隔線-------------------------------------')

# 關閉 WebDriver
driver.quit()

Clicked 'I Accept' button.
Clicked 'Load More' button 1 time(s).
Clicked 'Load More' button 2 time(s).
Clicked 'Load More' button 3 time(s).
len(transcript_links):  240
transcript_url:  https://www.fool.com/earnings/call-transcripts/2024/05/14/stoneco-stne-q1-2024-earnings-call-transcript/
stock_symbol: STNE
header_text: StoneCo (STNE) Q1 2024 Earnings Call Transcript
date_text: May 13, 2024
time_text: 5:00 p.m. ET
我是分隔線-------------------------------------
transcript_text Founded in 1993, The Motley Fool is a financial services company dedicated to making the world smarter, happier, and richer. The Motley Fool reaches millions of people every month through our premium investing solutions, free guidance and market analysis on Fool.com, top-rated podcasts, and non-profit The Motley Fool Foundation.

Founded in 1993, The Motley Fool is a financial services company dedicated to making the world smarter, happier, and richer. The Motley Fool reaches millions of people every month through ou

In [43]:
new_row = pd.DataFrame([{
    'stock_symbol': stock_symbol,  # 你需要從抓取的資料中找到這個欄位的內容
    'title': header_text,
    'date': date_text,
    'paragraphs': transcript_text
}])
df = pd.concat([df, new_row], ignore_index=True)
# 將日期轉換為適當的格式
df['date'] = pd.to_datetime(df['date'])
df

,stock_symbol,title,date,paragraphs
0,TTWO,Take-Two Interactive Software (TTWO) Q4 2024 E...,2024-05-16,"Founded in 1993, The Motley Fool is a financia..."
1,BIDU,Baidu (BIDU) Q1 2024 Earnings Call Transcript,2024-05-16,"Founded in 1993, The Motley Fool is a financia..."
2,DT,Dynatrace (DT) Q4 2024 Earnings Call Transcript,2024-05-15,"Founded in 1993, The Motley Fool is a financia..."
3,STNE,StoneCo (STNE) Q1 2024 Earnings Call Transcript,2024-05-13,"Founded in 1993, The Motley Fool is a financia..."


In [20]:
df['date'][0]

Timestamp('2024-05-16 00:00:00')

In [35]:
import pandas as pd
import yfinance as yf

# 讀取 DataFrame
test_df = df.copy()

# 設定 Yahoo Finance 的股票代號
stock_symbol = 'TTWO'

# 設定查詢日期
start_date = '2023-05-13'  # 這是你想要查找的 Yahoo Finance 的日期，通常為新聞日期的下一天
end_date = '2023-05-15'

try:
    # 查找 Yahoo Finance 上的歷史股價
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    print(stock_data)
except Exception as e:
    print(f"在查找股價時出現錯誤: {e}")

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TTWO']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-05-13 -> 2023-05-15)')


$TTWO: possibly delisted; No price data found  (1d 2023-05-13 -> 2023-05-15)
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


In [29]:
import yfinance as yf
import pandas as pd

# 設定股票代碼和時間範圍
ticker_symbol = 'TTWO'
start_date = '2020-01-01'
end_date = '2023-12-31'

# 抓取歷史股價資料
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# 顯示資料
print(stock_data)

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-02  123.529999  123.980003  121.400002  122.080002  122.080002   
2020-01-03  120.440002  122.500000  120.279999  122.209999  122.209999   
2020-01-06  122.019997  125.129997  121.550003  124.690002  124.690002   
2020-01-07  124.589996  126.470001  123.669998  125.510002  125.510002   
2020-01-08  125.959999  129.000000  124.820000  127.820000  127.820000   
...                ...         ...         ...         ...         ...   
2023-12-22  158.880005  162.899994  158.880005  161.949997  161.949997   
2023-12-26  162.000000  162.259995  161.160004  161.619995  161.619995   
2023-12-27  161.589996  161.729996  160.500000  160.990005  160.990005   
2023-12-28  160.990005  162.250000  160.399994  161.470001  161.470001   
2023-12-29  161.500000  162.919998  160.720001  160.949997  160.949997   

             Volume  
Date           

In [6]:
import pandas as pd
raw_data = pd.read_excel('Kaggle_NASDAQ_10_2016_To_2020.xlsx')
raw_data

,Company,Date,Content
0,AAPL,2016-04-26,\n\nThomson Reuters StreetEvents Event Transcr...
1,AAPL,2016-01-26,\n\nThomson Reuters StreetEvents Event Brief\n...
2,AAPL,2016-07-26,\n\nThomson Reuters StreetEvents Event Brief\n...
3,AAPL,2016-10-25,\n\nThomson Reuters StreetEvents Event Brief\n...
4,AAPL,2017-08-01,\n\nThomson Reuters StreetEvents Event Brief\n...
...,...,...,...
183,NVDA,2019-05-16,\n\nThomson Reuters StreetEvents Event Transcr...
184,NVDA,2019-11-14,\n\nThomson Reuters StreetEvents Event Transcr...
185,NVDA,2020-08-19,\n\nRefinitiv StreetEvents Event Transcript\nE...
186,NVDA,2020-02-13,\n\nThomson Reuters StreetEvents Event Transcr...


In [7]:
raw_data['Content'][0]

"\n\nThomson Reuters StreetEvents Event Transcript\nE D I T E D   V E R S I O N\n\nQ2 2016 Apple Inc Earnings Call\nAPRIL 26, 2016 / 9:00PM GMT\n\n================================================================================\nCorporate Participants\n================================================================================\n\n * Luca Maestri\n   Apple Inc. - CFO\n * Tim Cook\n   Apple Inc. - CEO\n * Nancy Paxton\n   Apple Inc. - Senior Director of IR\n\n================================================================================\nConference Call Participiants\n================================================================================\n\n * Katy Huberty\n   Morgan Stanley - Analyst\n * Gene Munster\n   Piper Jaffray & Co. - Analyst\n * Rod Hall\n   JPMorgan - Analyst\n * Shannon Cross\n   Cross Research - Analyst\n * Toni Sacconaghi\n   Bernstein - Analyst\n * Simona Jankowski\n   Goldman Sachs - Analyst\n * Steve Milunovich\n   UBS - Analyst\n\n======================